

## Food Image Classification
Given images of 101 diffrent foods, classify the foods presented in a given image.
We will use a TensorFlow/Keras pretrained MobileNetV2 CNN to make our predictions.
## Getting Started

In [57]:
#For working with the data
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import random

#For visualisation
import matplotlib.pyplot as plt
import seaborn as sns

#For preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#For building the model
import tensorflow as tf

#For analyzing the performance
from sklearn.metrics import confusion_matrix, classification_report

In [58]:
image_dir = Path('../input/food41/images')

# Creating File DataFrame
for a dataframe to contain one column with all the path files and the other column with all the labels for each image associated with file path

In [59]:
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

#turns filepaths and labels into panda series and concatenates them into a data frame
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([filepaths, labels], axis=1) #along axis=1 -> side by side

#there are too many images for this tutorial, therefore we are gone just use 100 imageges
#for each class
category_samples = []
for category in images['Label'].unique():
    category_slice = images.query("Label == @category")
    category_samples.append(category_slice.sample(100, random_state=1))
#axis=0 -> on top of each other; frac=1.0 -> sample 100% of the data without replacement
image_df = pd.concat(category_samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [60]:
image_df

In [61]:
images.query("Label == 'apple_pie'")

In [62]:
image_df['Label'].value_counts()

## Print random images to see how the data looks like

In [65]:
# visualize the training data
W = 10
H = 10
fig, axes = plt.subplots(W, H, figsize = (20,20))

axes = axes.ravel() # flatten the matrix into array
# Select a random number from 0 to n_training/ images will be selected randomly
for i in np.arange(0, W * H): 
    # Select a class randomly
    label = random.choice(os.listdir(image_dir))
    class_dir = os.path.join(image_dir,label)
    # Select a random image
    image = random.choice(os.listdir(class_dir))
    # read and display an image with the selected index    
    img = plt.imread(os.path.join(class_dir,image))
    axes[i].imshow( img )
    #print(np.array(img).shape)
    axes[i].set_title(label, fontsize = 8) # the label
    axes[i].axis('off')


# Train-Test Split

In [66]:
train_df, test_df = train_test_split(image_df, train_size=0.8, shuffle=True, random_state=1)

# Creating Generators
for loading the images of one batch at a time for not running out of memory

In [67]:
# All images will be rescaled by 1./255.
# Applying some Augmentation
train_datagen = ImageDataGenerator( rescale = 1.0/255.,
                                  rotation_range=30 ,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  brightness_range=[0.6,1],
                                  fill_mode='nearest',
                                  validation_split=0.2)

test_datagen = ImageDataGenerator(rescale = 1.0/255.0)


In [68]:
#speciffies which images will be used, then the generator takes images through the
#data frame, trains on them and recycles them
train_images = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical', #since it is a multi-class classification task
    batch_size=32,
    shuffle=True,
    seed=42, #to reproduce results
    subset='training' #only available when a validation split is used, specifies if 
                    #20% validation or 80% traing is taken
)

val_images = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical', #since it is a multi-class classification task
    batch_size=32,
    shuffle=True,
    seed=42, #to reproduce results
    subset='validation' #only available when a validation split is used, specifies if 
                    #20% validation or 80% traing is taken
)

test_images = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical', #since it is a multi-class classification task
    batch_size=32,
    shuffle=False
)

In [69]:
# Print the classes
train_images.class_indices

# Visualize after Augmentation

In [70]:
# viualize some images after the augmentation
x_batch, y_batch = next(train_images)
W = 5
H = 5
fig, axes = plt.subplots(W, H, figsize = (17,17))

axes = axes.ravel() # flaten the matrix into array
for i in np.arange(0, W * H): #from 0 to 5x5=25

    # Select a random image
    image = x_batch[i]
    # read and display an image with the selected index    
    axes[i].imshow( image )
    axes[i].set_title(np.argmax(y_batch[i]), fontsize = 8) # the label
    axes[i].axis('off')

# Modeling

### with Transfer Learning using MobileNetV2

In [71]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False, #defines if the final classifaction layer that the original model was trained on will be kept or not;
    #imagenet has 1000 classes, what we do not want here now, therefore we put our own classifaction layer at the top of the model
    weights='imagenet',
    pooling='avg' #ensure the output is one-dimensional
)

pretrained_model.trainable = False #ensures that we don't mess up the original net weight

In [72]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

print(model.summary())

# Training

In [73]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', #categorical_crossentropy, becuase we used Generators
    metrics=['accuracy'] #accuracy, because multi-class
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            #when the validation loss stops improving for three consecutive epochs, we'll stop
            #training and restore the weights from the best epoch
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

# Results

In [74]:
results = model.evaluate(test_images, verbose=1) #verbose=0 ->to not get the loading bar
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [34]:
# Plotting Results
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'green', label='Training acc')
plt.plot(epochs, val_acc, 'orange', label='Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.title('Training and validation accuracy')
plt.legend()
fig = plt.figure()
fig.savefig('acc.png')


plt.plot(epochs, loss, 'green', label='Training loss')
plt.plot(epochs, val_loss, 'orange', label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and validation loss')

plt.legend()
plt.show()

In [35]:
predictions = np.argmax(model.predict(test_images), axis=1)

cm = confusion_matrix(test_images.labels, predictions)
clr = classification_report(test_images.labels, predictions, target_names=test_images.class_indices, zero_division=0)

In [36]:
plt.figure(figsize=(30, 30))
sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues', cbar=False)
plt.xticks(ticks=np.arange(101) + 0.5, labels=test_images.class_indices, rotation=90)
plt.yticks(ticks=np.arange(101) + 0.5, labels=test_images.class_indices, rotation=0)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [37]:
print("Classification Report:\n----------------------\n", clr)